In [1]:
import os
HOME = os.getcwd()
print(HOME)

/home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt


In [3]:
# !mkdir {HOME}/images
# !mkdir {HOME}/videos

In [4]:
# !pip install moviepy

### Splitting video into test and train

In [5]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

# Define the path to your input video
input_video_path = "raw_video/conveyor_chocolate.mp4"

# Define the start and end times for the first split (in seconds)
split1_start_time = 0
split1_end_time = 14

# Define the start and end times for the second split (in seconds)
split2_start_time = 14
split2_end_time = 18

# Define the paths for the output videos
output_video1_path = "videos/video1.mp4"
output_video2_path = "videos/video2.mp4"

# Split the first part of the video
ffmpeg_extract_subclip(input_video_path, split1_start_time, split1_end_time, targetname=output_video1_path)

# Split the second part of the video
ffmpeg_extract_subclip(input_video_path, split2_start_time, split2_end_time, targetname=output_video2_path)


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


In [6]:
VIDEO_DIR_PATH = f"{HOME}/videos"
IMAGE_DIR_PATH = f"{HOME}/images"
FRAME_STRIDE = 2

### Creating images from video

In [7]:
import supervision as sv
from tqdm.notebook import tqdm

video_paths = sv.list_files_with_extensions(
    directory=VIDEO_DIR_PATH, 
    extensions=["mov", "mp4"])

TEST_VIDEO_PATHS, TRAIN_VIDEO_PATHS = video_paths[:1], video_paths[1:]

for video_path in tqdm(TRAIN_VIDEO_PATHS):
    video_name = video_path.stem
    image_name_pattern = video_name + "-{:05d}.png"
    with sv.ImageSink(target_dir_path=IMAGE_DIR_PATH, image_name_pattern=image_name_pattern) as sink:
        for image in sv.get_video_frames_generator(source_path=str(video_path), stride=FRAME_STRIDE):
            sink.save_image(image=image)

  0%|          | 0/1 [00:00<?, ?it/s]

### Augmentation

In [6]:
# !pip install Augmentor

In [14]:
import Augmentor
import os

# Define the path to the directory containing the images
input_directory = "images/"

# Define the path for the augmented output directory
output_directory = "images/"

# Create an Augmentor pipeline with the input and output directories
p = Augmentor.Pipeline(input_directory, '')

# Add the desired augmentation operations to the pipeline
# p.rotate(probability=0.5, max_left_rotation=10, max_right_rotation=10)
p.flip_left_right(probability=0.8)
p.zoom_random(probability=0.8, percentage_area=0.8)

# Specify the number of augmented images to generate
num_augmented_images = 15

# Augment the images
p.sample(num_augmented_images)

# Rename the augmented images to remove the "_augmented" suffix
for filename in os.listdir(output_directory):
    if filename.endswith("_augmented.png"):  # Adjust the file extension if necessary
        new_filename = filename.replace("_augmented", "")
        os.rename(os.path.join(output_directory, filename),
                  os.path.join(output_directory, new_filename))


Initialised with 15 image(s) found.
Output directory set to images/.

Processing <PIL.Image.Image image mode=RGB size=1280x720 at 0x7F84DE15EFE0>: 100%|██████████| 15/15 [00:00<00:00, 15.79 Samples/s]


### Resizing image to prevent cuda out of memory

In [9]:
from PIL import Image
import os

def resize_images(source_dir, target_dir, new_size):
    # Create the target directory if it doesn't exist
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Get the list of files in the source directory
    files = os.listdir(source_dir)

    for file_name in files:
        # Construct the full path to the source file
        source_file = os.path.join(source_dir, file_name)

        # Load the image
        image = Image.open(source_file)

        # Resize the image
        resized_image = image.resize(new_size)

        # Construct the full path to the target file
        target_file = os.path.join(target_dir, file_name)

        # Save the resized image to the target directory
        resized_image.save(target_file)

        print(f"Resized and saved: {target_file}")

# Example usage
source_directory = f"{HOME}/images"
target_directory = f"{HOME}/images_resized"
new_size = (800, 600)

resize_images(source_directory, target_directory, new_size)


Resized and saved: /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images_resized/images_original_video2-00012.png_b5078a94-845b-49b6-8c02-d9cd60b288a2.png
Resized and saved: /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images_resized/video2-00005.png
Resized and saved: /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images_resized/video2-00013.png
Resized and saved: /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images_resized/video2-00011.png
Resized and saved: /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images_resized/images_original_video2-00014.png_e5bc144a-377f-47aa-804f-3686e366db2a.png
Resized and saved: /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images_resized/images_original_video2-00004.png_b3fa8fb8-cf21-4da3-9742-071eaf8f2abb.png
Resized and saved: /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images_resized/images_original_video2-00011.png_4a30881b-a06a-4fae-8036-1b1864b94882.png
Re

### Helper function to shortening Image names

In [11]:
import os

def shorten_image_names(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".png"):
            old_name = os.path.join(directory, filename)
            new_name = os.path.join(directory, filename[-10:])
            os.rename(old_name, new_name)
            print(f"Renamed: {old_name} -> {new_name}")


# Example usage
directory = f"{HOME}/images"
shorten_image_names(directory)


Renamed: /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images/images_original_video2-00012.png_b5078a94-845b-49b6-8c02-d9cd60b288a2.png -> /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images/b288a2.png
Renamed: /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images/video2-00005.png -> /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images/-00005.png
Renamed: /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images/video2-00013.png -> /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images/-00013.png
Renamed: /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images/video2-00011.png -> /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images/-00011.png
Renamed: /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images/images_original_video2-00014.png_e5bc144a-377f-47aa-804f-3686e366db2a.png -> /home/darth/MLAI-projects/yolo_conveyor/yolo_conveyor_belt/images/66db2a.png
Renamed: /home/darth/MLAI-